# **TP N°1 Représentation de l'Information : Indexation**

*importer les bibliotheque necessaire*

In [1]:
import os
import nltk
from nltk.tokenize import RegexpTokenizer
from collections import defaultdict
import json


In [2]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ryan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


*definir le path de nos collections*

In [3]:
# Crée la liste des documents dans le dossier "Collection"
collection_path = '../Collections'  
documents = [f for f in os.listdir(collection_path) if f.endswith('.txt')]

In [4]:
def extraire_nom_sans_extension(fichier):
    return fichier.split('.')[0]

In [5]:

ExpReg = RegexpTokenizer('(?:[A-Z]\.)+|\d+|\w+|\.{3}')
# Cette partie permet de capturer des abréviations comme "D.Z.", "U.S.A.", etc. Cela prend en compte une ou plusieurs lettres majuscules suivies d'un point.
# Cela permet de capturer des nombres comme 123, 45, 7890, etc.
# Cela permet de capturer des mots comme "apple", "hello", "AI123", etc.
# Cette partie capture des points de suspension, c'est-à-dire "...".

stop_words = set(stopwords.words('english'))
porter_stemmer = nltk.PorterStemmer()
lancaster_stemmer = nltk.LancasterStemmer()

*Extraction automatique des termes*

In [6]:
def extraire_termes(texte):
    termes_split = texte.split()  # Utilisation de split()
    termes_regexp = ExpReg.tokenize(texte)  # Utilisation de RegexpTokenizer
    return termes_split, termes_regexp

*Suppression des mots-vides*

In [7]:
def supprimer_mots_vides(termes):
    return [terme for terme in termes if terme.lower() not in stop_words]

*Normalisation (stemming) des termes extraits*

* **Lancaster Stemmer** : Il est généralement plus agressif et a tendance à réduire les mots de manière plus radicale. Par exemple, il peut couper des parties importantes des mots et produire des formes très courtes qui peuvent être difficiles à interpréter. Cela peut parfois nuire à la précision dans certaines applications.
* **Porter Stemmer** : Il est moins agressif et produit des formes plus proches de la racine réelle des mots. Le Porter Stemmer est conçu pour être moins destructeur que Lancaster et est généralement plus adapté pour les applications courantes de recherche d'information.

In [8]:
def normaliser_termes(termes):
    termes_porter = [porter_stemmer.stem(terme) for terme in termes]
    termes_lancaster = [lancaster_stemmer.stem(terme) for terme in termes]
    return termes_porter, termes_lancaster

*Génération des fichier descripteurs*

In [9]:
# Fonction pour générer le fichier descriptif
def generer_fichier_descripteur(doc, termes_porter ,termes_lancaster):
    descripteur_path1 = '..\Descripteurs\porter'
    descripteur_path2 = '..\Descripteurs\lancaster'
    
    os.makedirs(descripteur_path1, exist_ok=True)
    os.makedirs(descripteur_path2, exist_ok=True)
    
    descripteur_file1 = os.path.join(descripteur_path1, f"{doc}_descripteur.txt")
    descripteur_file2 = os.path.join(descripteur_path2, f"{doc}_descripteur.txt")
    
    nom_sans_extension = extraire_nom_sans_extension(doc)
    
    with open(descripteur_file1, 'w', encoding='utf-8') as f:
        for terme in termes_porter:
            f.write(f"{nom_sans_extension} {terme}\n")
       
    with open(descripteur_file2, 'w', encoding='utf-8') as f:
        for terme in termes_lancaster:
            f.write(f"{nom_sans_extension} {terme}\n")
       

*Construire le fichier inverse*

In [10]:
def construire_index_inverse(document , termes_porter , termes_lancaster ,index_inverse_porter , index_inverse_lancaster ): #pour un doument
    nom_sans_extension = extraire_nom_sans_extension(document)
            
    for terme in termes_porter:  # Utiliser set pour éviter les doublons
            index_inverse_porter.setdefault(terme, []).append(nom_sans_extension)

    for terme in termes_lancaster:  # Utiliser set pour éviter les doublons set(termes_lancaster)
        index_inverse_lancaster.setdefault(terme, []).append(nom_sans_extension)

In [11]:
index_inverse_porter = {}
index_inverse_lancaster = {}

for doc in documents:
    with open(os.path.join(collection_path, doc), 'r', encoding='utf-8') as file:
        texte = file.read()
        
    nom_sans_extension = extraire_nom_sans_extension(doc)
    
    # Étape 1: Extraction des termes
    termes_split, termes_regexp = extraire_termes(texte)
    print(f"Document {nom_sans_extension} - Split (): {termes_split}")
    print(f"Document {nom_sans_extension} - RegexpTokenizer(): {termes_regexp}")
    print(f" length : {len(termes_regexp)}")

    # Étape 2: Suppression des mots vides
    termes_sans_mots_vides = supprimer_mots_vides(termes_regexp)
    print(f"Document {nom_sans_extension} - termes sans mots vides: {termes_sans_mots_vides}")

    # Étape 3: Normalisation avec PorterStemmer/LancerStemmer
    termes_porter, termes_lancaster = normaliser_termes(termes_sans_mots_vides)
    print(f"Document {nom_sans_extension} - PorterStemmer: {termes_porter}")

    # Étape 5: Génération du fichier descriptif
    generer_fichier_descripteur(doc, termes_porter ,termes_lancaster)
    construire_index_inverse(doc , termes_porter, termes_lancaster ,index_inverse_porter , index_inverse_lancaster)
    



Document D1 - Split (): ['experimental', 'investigation', 'of', 'the', 'aerodynamics', 'of', 'a', 'wing', 'in', 'a', 'slipstream', '.', 'an', 'experimental', 'study', 'of', 'a', 'wing', 'in', 'a', 'propeller', 'slipstream', 'was', 'made', 'in', 'order', 'to', 'determine', 'the', 'spanwise', 'distribution', 'of', 'the', 'lift', 'increase', 'due', 'to', 'slipstream', 'at', 'different', 'angles', 'of', 'attack', 'of', 'the', 'wing', 'and', 'at', 'different', 'free', 'stream', 'to', 'slipstream', 'velocity', 'ratios', '.', 'the', 'results', 'were', 'intended', 'in', 'part', 'as', 'an', 'evaluation', 'basis', 'for', 'different', 'theoretical', 'treatments', 'of', 'this', 'problem', '.', 'the', 'comparative', 'span', 'loading', 'curves,', 'together', 'with', 'supporting', 'evidence,', 'showed', 'that', 'a', 'substantial', 'part', 'of', 'the', 'lift', 'increment', 'produced', 'by', 'the', 'slipstream', 'was', 'due', 'to', 'a', '/destalling/', 'or', 'boundary-layer-control', 'effect', '.', 'th

In [55]:
# Chemin pour l'index inversé
index_inverse_path = '../Inverse/index_inverse.json'

# Créer le répertoire s'il n'existe pas
os.makedirs(os.path.dirname(index_inverse_path), exist_ok=True)

index_inverse = {
    'Porter': index_inverse_porter,
    'Lancaster': index_inverse_lancaster
}

# Sauvegarder les index dans le fichier JSON
with open(index_inverse_path, 'w', encoding='utf-8') as f:
    json.dump(index_inverse, f, ensure_ascii=False, indent=4)

print(f"L'index inversé a été sauvegardé dans {index_inverse_path}.")



L'index inversé a été sauvegardé dans ../Inverse/index_inverse.json.
